In [1]:
import sys
import time
import json
from pprint import pprint

import pandas as pd

from neo4j import GraphDatabase
from dask import delayed, compute
from dask.distributed import Client

import helpers.graph_services as gs
import helpers.helper as helper

In [2]:
def check_if_graph_exists(neo4j_connection, graph_name):
    query="""CALL gds.graph.exists('{graph_name}')""".format(graph_name=graph_name)
    res=neo4j_connection.run_single_query(query)
    return res

def get_graph_information(neo4j_connection, graph_name):
    query="""CALL gds.graph.list('{graph_name}')""".format(graph_name=graph_name)
    res=neo4j_connection.run_single_query(query)
    return res

def create_graph(neo4j_connection, graph_name):
    query = """
    CALL gds.graph.create(
      '{graph_name}', 
      {{
        TWITCH_USER: {{
            label:'TWITCH_USER', 
            properties: ['days', 'views']
            }}
      }}, 
      '*'
    )
    """.format(graph_name=graph_name)
    res=neo4j_connection.run_single_query(query)
    return res

In [3]:
GRAPH_NAME="twitch-graph"

In [4]:
neo4j_connection = gs.graph_driver(uri_scheme='bolt', host='localhost', port='8687', username='neo4j', password='vaibhav123')
existing_graphs = check_if_graph_exists(neo4j_connection, GRAPH_NAME)
if not existing_graphs[0]['exists']:
    r = create_graph(neo4j_connection, GRAPH_NAME)
else:
    r = get_graph_information(neo4j_connection, GRAPH_NAME)
res = dict(r[0].items())
pprint(res)


{'creationTime': neo4j.time.DateTime(2021, 1, 14, 3, 2, 19.080034, tzinfo=<StaticTzInfo 'Etc/UTC'>),
 'database': 'neo4j',
 'degreeDistribution': {'max': 1491,
                        'mean': 12.586166471277842,
                        'min': 0,
                        'p50': 4,
                        'p75': 12,
                        'p90': 28,
                        'p95': 45,
                        'p99': 122,
                        'p999': 486},
 'density': 0.0003688902509240553,
 'graphName': 'twitch-graph',
 'memoryUsage': '5764 KiB',
 'modificationTime': neo4j.time.DateTime(2021, 1, 14, 3, 2, 20.49528, tzinfo=<StaticTzInfo 'Etc/UTC'>),
 'nodeCount': 34120,
 'nodeProjection': {'TWITCH_USER': {'label': 'TWITCH_USER',
                                    'properties': {'days': {'defaultValue': None,
                                                            'property': 'days'},
                                                   'views': {'defaultValue': None,
                 